In [55]:
import tensorflow as tf
import sys
import numpy as np
import json
sys.path.append('/Users/robertsonwang/Desktop/models/research/')
from object_detection.utils import dataset_util
from PIL import Image
import cv2
import json

In [59]:
def make_json_bboxes(filename, file_path, label = '8.0-9.0', output_path = ''):
	with open(file_path + filename, 'r') as read_file:
	    bboxes = read_file.read()
	bboxes = bboxes.replace(':'+label, '')
	clean_list = [x + ')]' for x in bboxes.split(')]') if x != '']

	dict_key = []
	dict_list = []

	for item in test_list:
	    tup = item.split(':')
	    dict_key.append(tup[0].encode('utf-8'))
	    dict_list.append(eval(tup[1]))
	final_dict = {k:v for k,v in zip(dict_key, dict_list)}

	with open(output_path + filename.split('.')[0] + '.json', 'w') as outfile:
	    json.dump(final_dict, outfile)

In [60]:
make_json_bboxes('bound_results_train.txt', '/Users/robertsonwang/Desktop/bb_images/data/')

In [2]:
def create_tf_example(encoded_ts, filename, xmin, xmax, ymin, ymax):
#   Creates a tf proto from sample test_strip image.
#   Args:
#     encoded_ts: The jpg encoded data of the test_strip image.
#   Returns:
#     example: The created tf.Example.

    height = 250
    width = 250
    image_format = b'jpg'

    xmins = [float(xmin)/float(width)]
    xmaxs = [float(xmax)/float(width)]
    ymins = [float(ymin)/float(height)]
    ymaxs = [float(ymax)/float(height)]

    classes_text = ['test_strip']
    classes = [1]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_ts),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    
    return tf_example

In [3]:
def load_image(addr):
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.imread(addr)
    img = cv2.resize(img, (250, 250), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img

def importImage(fileName):
    imgText = open(fileName, 'rb')
    imgTextStr = imgText.read()
    imgText.close()
    return imgTextStr

In [4]:
file_name = 'IMG_0896.JPG'
test_image = importImage(file_name)
xmin = 81.0
xmax = 138.0
ymin = 97.0
ymax = 153.0
test = create_tf_example(test_image, file_name, xmin, xmax, ymin, ymax)

In [14]:
bboxes = {"/Users/robertsonwang/Desktop/bb_images/IMG_0896.JPG":[(81, 97), (138, 153)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0897.JPG":[(77, 101), (126, 142)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0898.JPG":[(80, 60), (151, 125)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0899.JPG":[(104, 91), (152, 147)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0900.JPG":[(61, 95), (104, 177)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0901.JPG":[(88, 87), (135, 139)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0902.JPG":[(71, 117), (117, 155)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0903.JPG":[(93, 85), (169, 162)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0904.JPG":[(78, 87), (174, 156)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0905.JPG":[(75, 81), (169, 164)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0906.JPG":[(99, 105), (180, 176)], 
          "/Users/robertsonwang/Desktop/bb_images/IMG_0908.JPG":[(84, 62), (164, 140)]}

In [12]:
def main(_):
    writer = tf.python_io.TFRecordWriter('/Users/robertsonwang/Desktop/bb_images/shards/')

    for image in bboxes.keys():
        image_jpg = importImage(image)
        xmin, xmax, ymin, ymax = float(bboxes[0][0]), float(bboxes[1][0]), \
                                float(bboxes[0][1]), float(bboxes[1][1])
        tf_example = create_tf_example(image_jpg, image, xmin, xmax, ymin, ymax)
        tf_example = create_tf_example(example)
        writer.write(tf_example.SerializeToString())

    writer.close()

In [13]:
tf.app.run()

FailedPreconditionError: /Users/robertsonwang/Desktop/bb_images/shards/